In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv, global_mean_pool
from torch_geometric.data import Data
#from graph_builder import GraphBuilder  # <-- External builder
import pandas as pd
from torch.nn import Linear, ReLU, Sequential
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_add_pool
from sklearn.model_selection import train_test_split
import ast
from torch_geometric.utils import degree


First we read the edges and coefficients of the csv files and save them in lists.

Here we read the files for 5 to 8 loops.

We aim to use 9 and 10 loop data for testing.

In [2]:
edges=[]
y=[]
for i in range(5, 9):
    filename = f'../Graph_Edge_Data/den_graph_data_{i}.csv'
    df = pd.read_csv(filename)
    edges += df['EDGES'].tolist()
    y += df['COEFFICIENTS'].tolist()

In [3]:
edges = [ast.literal_eval(e) for e in edges]

In [4]:
from torch_geometric.transforms import AddLaplacianEigenvectorPE
eigen_vec= AddLaplacianEigenvectorPE(k=3,attr_name=None)

We need to now translate the edges into dataset forms for training and testing.

In [5]:
class GraphBuilder:
    def __init__(self, solid_edges, coeff, node_labels=None):
        # Auto-infer node labels if not provided
        if node_labels is None:
            node_labels = sorted(set(u for e in solid_edges for u in e))
        self.node_labels = node_labels
        self.label2idx = {label: i for i, label in enumerate(node_labels)}

        self.solid_edges = solid_edges
        self.num_nodes = len(self.node_labels)
        self.y = torch.tensor(coeff, dtype=torch.long)  # Ensure y is a column vector

    def build(self, extra_node_features=None):
        edge_list = []

        for u, v in self.solid_edges:
            i, j = self.label2idx[u], self.label2idx[v]
            edge_list += [[i, j], [j, i]]  # bidirectional

        edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()

        # Basic node feature: degree
        degree_feat = degree(edge_index[0], num_nodes=self.num_nodes).view(-1, 1)

        # Combine degree with extra features if provided
        if extra_node_features is not None:
            assert extra_node_features.shape[0] == self.num_nodes, \
                "extra_node_features must match number of nodes"
            x = torch.cat([degree_feat, extra_node_features], dim=1)
        else:
            x = degree_feat
        return Data(x=x, edge_index=edge_index, num_nodes=self.num_nodes, coeff=self.y)


In [6]:
data=[GraphBuilder(solid_edges=x,coeff=y0).build() for x,y0 in zip(edges,y)]
data = [eigen_vec(d) for d in data]

In [11]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=43)

In [12]:
# Combine graph_list and y into a DataLoader where graph_list represents x and y represents y with train/test split
train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)

We are interested in graph classification of 0 and 1. We add two graph convolutional layers, making sure that the message passing is extended to two neighbours, and then add graph pooling to average over the whole graph.

In [13]:
class SimpleGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch # For multiple graphs in a batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

In [14]:
# Create the model object using CNN class
model = SimpleGNN(in_channels=4, hidden_channels=200)

In [21]:
# import cross entropy loss
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum=0.005)

In [19]:
def train_model(model, train_loader, test_loader, optimizer, criterion, device, n_epochs=20):
    accuracy_list = []
    loss_list = []

    model.to(device)

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            out = model(batch)             # out = model(batch) handles batch.x, batch.edge_index, etc.
            loss = criterion(out, batch.coeff) # Use batch.y (or batch.coeff if that's what your dataset uses)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        loss_list.append(total_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch)
                _, predicted = torch.max(out, 1)
                correct += (predicted == batch.coeff).sum().item()
                total += batch.num_graphs  # If doing graph-level classification
                # total += batch.y.size(0)  # If doing node-level classification

        accuracy = correct / total
        accuracy_list.append(accuracy)

        print(f"Epoch {epoch+1}: Loss={total_loss:.4f}, Accuracy={accuracy:.4f}")


In [22]:
train_model(model,train_loader,test_loader,optimizer,n_epochs=100,device='cpu',criterion=criterion)

Epoch 1: Loss=200.7166, Accuracy=0.5488
Epoch 2: Loss=199.8406, Accuracy=0.5427
Epoch 3: Loss=198.9995, Accuracy=0.5915
Epoch 4: Loss=197.8695, Accuracy=0.5427
Epoch 5: Loss=198.0695, Accuracy=0.5427
Epoch 6: Loss=198.2646, Accuracy=0.5854
Epoch 7: Loss=197.4068, Accuracy=0.6159
Epoch 8: Loss=197.5364, Accuracy=0.5915
Epoch 9: Loss=197.2759, Accuracy=0.5854
Epoch 10: Loss=196.8274, Accuracy=0.5427
Epoch 11: Loss=197.2086, Accuracy=0.6159
Epoch 12: Loss=196.3848, Accuracy=0.6159
Epoch 13: Loss=197.2735, Accuracy=0.5854
Epoch 14: Loss=196.9252, Accuracy=0.5427
Epoch 15: Loss=196.2797, Accuracy=0.6098
Epoch 16: Loss=196.7048, Accuracy=0.5427
Epoch 17: Loss=196.6874, Accuracy=0.6159
Epoch 18: Loss=196.6030, Accuracy=0.5671
Epoch 19: Loss=196.4089, Accuracy=0.5732
Epoch 20: Loss=197.1290, Accuracy=0.5976
Epoch 21: Loss=196.5935, Accuracy=0.5427
Epoch 22: Loss=196.2088, Accuracy=0.5976
Epoch 23: Loss=196.1308, Accuracy=0.6037
Epoch 24: Loss=196.2427, Accuracy=0.5854
Epoch 25: Loss=196.5165, 